In [1]:
#importing packages
import bs4
from selenium import webdriver
from urllib.request import urlopen as uReq
import requests
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import time

In [2]:
#using beautiful soup to parse html
#replace "my_url" with the project url
my_url = 'https://www.kickstarter.com/projects/jamesreads/the-cosma-visions-oracle-deck-and-prisma-visions-tarot-book'
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,"html.parser")

In [3]:
#grabbing the title of the project
project_title = page_soup.find("h2",{"type-28 type-24-md soft-black mb1 project-name"}).text
project_title

'The Cosma Visions Oracle Deck & Prisma Visions Tarot Book'

In [4]:
#finding all pledge info
pledge = page_soup.findAll("div",{"class":"pledge__info"})
print(len(pledge))

31


In [5]:
#pledge[0] will have no information in any project,checking the html
pledge[1]

<div class="pledge__info">
<h2 class="pledge__amount">
Pledge <span class="money">$25</span> or more
<span class="pledge__currency-conversion">
About <span>$25</span>
</span>
</h2>
<h3 class="pledge__title">
The BIG VISIONS BOOK
</h3>
<div class="pledge__reward-description pledge__reward-description--expanded">
<p>An expanded book on reading Cosma Visions, Prisma Visions and Light Visions. With reversed meanings for all cards and an in-depth look at creating the decks with full page color images of all 156 cards. Over 400 pages of content, includes a free bookmark of the Cosmic Tree and the applicable stretch goal goodies!</p>
<span class="itemization-includes text-uppercase c-navy-light type-10 medium ls1px block">Includes:</span>
<ul class="type-14 ml3 green-dark mb2 pl2">
<li class="list-disc">
The Big Visions Hardcover Book
</li>
<li class="list-disc">
Cosmic Tree Bookmark
</li>
<li class="list-disc">
Cosma Visions Pin
</li>
<li class="list-disc">
Stretch Goal Stickers
</li>
</ul>


In [6]:
#looping through all the pledge infos and saving it into a dataframe
headers = ["Pledge_Amount", "Estimated_Delivery_Time","Backer_Count"]
df = pd.DataFrame(columns = headers)

if pledge[0].find("h2",{"class":"pledge__amount"}).text == '\nPledge without a reward\n':
    for info in pledge[1:len(pledge)]:
        pledge_amount = info.find("span","pledge__currency-conversion").span.text
        estimated_time_info = info.find("time","invisible-if-js js-adjust-time").text
        pledge_backer_count = re.search("\d+",info.find("span","pledge__backer-count").text).group(0)
        row = pd.Series([pledge_amount,estimated_time_info,pledge_backer_count],index = headers)
        df = df.append(row,ignore_index = True)
    
else:
    for info in pledge:
        pledge_amount = info.find("span","pledge__currency-conversion").span.text
        estimated_time_info = info.find("time","invisible-if-js js-adjust-time").text
        pledge_backer_count = re.search("\d+",info.find("span","pledge__backer-count").text).group(0)
        row = pd.Series([pledge_amount,estimated_time_info,pledge_backer_count],index = headers)
        df = df.append(row,ignore_index = True)




In [7]:
#backer information dataframe
df

,Pledge_Amount,Estimated_Delivery_Time,Backer_Count
0,$25,Feb 2021,7
1,$40,Dec 2020,19
2,US$ 40,Feb 2021,46
3,$50,Feb 2021,20
4,US$ 65,Feb 2021,40
5,$65,Feb 2021,89
6,US$ 80,Feb 2021,11
7,$85,Feb 2021,45
8,$90,Feb 2021,42
9,$100,Feb 2021,186


In [8]:
#grabbing the href for the updates page
update_page_link = page_soup.find("a",{"class":"js-load-project-content js-load-project-updates mx3 project-nav__link--updates tabbed-nav__link type-14"})["href"]
update_page_link = "https://www.kickstarter.com" + update_page_link
update_page_link

'https://www.kickstarter.com/projects/jamesreads/the-cosma-visions-oracle-deck-and-prisma-visions-tarot-book/posts'

In [9]:
#setting up selenium driver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(update_page_link)
driver.implicitly_wait(10)
time.sleep(5)

In [10]:
#Checking for the see more button
from selenium.common.exceptions import NoSuchElementException

try:
    element = driver.find_element_by_xpath('//*[@id="project-post-interface"]/div/div[11]/div/div/div/button')
    driver.execute_script("arguments[0].click();", element)
    driver.implicitly_wait(10)
    time.sleep(5)
    html_update = driver.page_source
    page_soup_updates = soup(html_update,"html.parser")
except NoSuchElementException:
    html_update = driver.page_source
    page_soup_updates = soup(html_update,"html.parser") 

In [11]:
#checking the number of updates
updates = page_soup_updates.findAll("a",{"class":"truncated-post soft-black block border border-grey-500 hover-border-dark-grey-400"})
number_of_updates = len(updates)
number_of_updates

14

In [12]:
#checking the number of updates visible only for backers
backer_updates = page_soup_updates.findAll("div",{"class":"truncated-post soft-black block border border-grey-500 hover-border-dark-grey-400"})
number_of_backer_updates = len(backer_updates)
number_of_backer_updates

2

In [13]:
#making a dataframe for the updates information
updates_headers = ["Number","Title","Date","Number_of_comments","Number_of_likes"]
df_updates = pd.DataFrame(columns = updates_headers)

for update in updates:
    update_number = update.find("span",{"class":"type-13 soft-black_50 text-uppercase"}).text
    update_title = update.find("h2",{"class":"mb3"}).text
    update_date = update.find("span",{"class":"type-13 soft-black_50 block-md"}).text
    
    if not update.findAll("span",{"class":"mr4"})[0].text:
        comments = 0
    else:    
        comments = update.findAll("span",{"class":"mr4"})[0].text
    
    if not update.findAll("span",{"class":"mr4"})[1].text:
        likes = 0
    else:
        likes = update.findAll("span",{"class":"mr4"})[1].text
        
    row = pd.Series([update_number,update_title,update_date,comments,likes],index = updates_headers)
    df_updates = df_updates.append(row,ignore_index = True)

for update in backer_updates:
    update_number = update.find("span",{"class":"type-13 soft-black_50 text-uppercase"}).text
    update_title = update.find("h2",{"class":"mb3"}).text
    update_date = update.find("span",{"class":"type-13 soft-black_50 block-md"}).text
    
    if not update.findAll("span",{"class":"mr4"})[0].text:
        comments = 0
    else:    
        comments = update.findAll("span",{"class":"mr4"})[0].text
    
    if not update.findAll("span",{"class":"mr4"})[1].text:
        likes = 0
    else:
        likes = update.findAll("span",{"class":"mr4"})[1].text
        
    row = pd.Series([update_number,update_title,update_date,comments,likes],index = updates_headers)
    df_updates = df_updates.append(row,ignore_index = True)

    
df_updates

,Number,Title,Date,Number_of_comments,Number_of_likes
0,Update #15,What Comes Next!,"August 28, 2020",78,186
1,Update #14,Cosma Visions DELUXE!,"August 27, 2020",83,132
2,Update #13,An Update on Shipping,"August 26, 2020",53,178
3,Update #12,Sample Bookmarks are in!,"August 25, 2020",21,166
4,Update #11,A look at our next stretch goal: Cosma Visions...,"August 24, 2020",40,159
5,Update #10,Stretch Goal Unlocked: Cosmic Tree Bookmark,"August 21, 2020",36,188
6,Update #9,The 79th Card of Cosma Visions,"August 20, 2020",55,179
7,Update #8,Visuals of all the Pledge Tiers,"August 19, 2020",12,94
8,Update #6,Stretch Goal Unlocked: Past Lives Pin,"August 17, 2020",17,119
9,Update #5,Stretch Goals!!,"August 14, 2020",105,138


In [14]:
#clicking on comments tab
element_comments = driver.find_element_by_xpath('//*[@id="comments-emoji"]')
driver.execute_script("arguments[0].click();", element_comments)
driver.implicitly_wait(10)

In [15]:
#clicking on show more comments until it shows all
condition = True
while condition:
    try:
        element = driver.find_element_by_xpath('//*[@id="react-project-comments"]/div/button')
        driver.execute_script("arguments[0].click();", element)
        driver.implicitly_wait(10)
        time.sleep(5)
    except NoSuchElementException:
        condition = False 

In [16]:
#clicking on show more replies until it shows all
condition = True
while condition:
    try:
        element = driver.find_element_by_css_selector('.bttn.bttn-medium.bttn-white.flex.w100p.keyboard-focusable')
        driver.execute_script("arguments[0].click();", element)
        driver.implicitly_wait(10)
        time.sleep(5)
    except NoSuchElementException:
        condition = False 


In [17]:
#parsing the comments page
html_comments = driver.page_source
page_soup_comments = soup(html_comments,"html.parser") 

In [18]:
page_comments = page_soup_comments.findAll("div",{"class":"border-box relative break-word border border-grey-400 px3 pt3 pb2 o100p transition-all transition-delay-1000 bg-white"})
total_comments = len(page_comments)
total_comments

673

In [19]:
#making a dataframe for the comments section
comments_headers = ["UserName","Comment","Status","Time"]
df_comments = pd.DataFrame(columns = comments_headers)

for comment in page_comments:
    user_name = comment.find("span",{"class":"mr2"}).text
    user_comment = comment.find("p",{"class":"type-14 mb0"}).text
    if not comment.findAll("span",{"class":"apricot-600 type-14 mr1"}):
        if not comment.findAll("span",{"class":"bg-ksr-green-700 white px1 type-14 mr1"}):
            user_status = "backer"
        else:
            user_status = comment.find("span",{"class":"bg-ksr-green-700 white px1 type-14 mr1"}).text
    else:
        user_status = comment.find("span",{"class":"apricot-600 type-14 mr1"}).text 
    user_time = comment.find("time",{"class":"block dark-grey-400 type-12"})["title"]
    
    row = pd.Series([user_name,user_comment,user_status,user_time],index = comments_headers)
    df_comments = df_comments.append(row,ignore_index = True)

In [20]:
df_comments

,UserName,Comment,Status,Time
0,Prochi,Omg just jumped on the bandwagon and went all ...,backer,"September 1, 2020 12:13 PM EDT"
1,James R. Eads,"Welcome, Prochi!",Creator,"September 1, 2020 1:18 PM EDT"
2,Elizabeth Nahum-Albright,Cheers-,backer,"September 1, 2020 10:17 AM EDT"
3,James R. Eads,Thanks for being here Lizzy!!,Creator,"September 1, 2020 1:19 PM EDT"
4,Tabitha,I've just pledged for the first time and added...,backer,"September 1, 2020 8:31 AM EDT"
...,...,...,...,...
668,Shelly Tannehill,Funded in 8 minutes! Congratulations!! It's ...,Superbacker,"August 11, 2020 1:09 PM EDT"
669,James R. Eads,my mind is blown!!,Creator,"August 11, 2020 1:21 PM EDT"
670,Smote,Would there be any chance to add the bonus car...,backer,"August 11, 2020 1:05 PM EDT"
671,James R. Eads,That's pretty unlikely. Those cards are what m...,Creator,"August 11, 2020 1:21 PM EDT"
